In [ ]:
import os
import numpy as np
from cmdstanpy import cmdstan_path, CmdStanModel

import arviz as az
import seaborn as sns
import matplotlib.pyplot as plt

from tensorflow_probability.substrates import numpy as tfp
tfd = tfp.distributions

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [ ]:
data = pd.read_csv('tf.csv')

In [ ]:
data_tfidf = pd.read_csv('tfidf.csv')

In [ ]:
dat = data.iloc[:,1:17094]
dat.shape

(2000, 17093)

In [ ]:
dat.head()

,distanc,well-equip,concur,goos,left-field,matrix,neurotransmitt,talbott,equip,relax,...,harmless,un-us,.becaus,bodywork,infatu,pratchett,awar,dude,jp,sector
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
list_of_words = dat.columns.values.tolist() # complete list of words

Remove words that apper less than 10 times in all documents

In [ ]:
num_words = dat.sum(axis = 0).to_numpy() #viene un array 17093 dimensionale
TOT = 80
idx = np.asarray(np.where(num_words > TOT))[0,:]
idx.shape

(415,)

In [ ]:
da = dat.iloc[:,idx]
da.head()

,shallow,still,portray,experienc,4,serv,cours,son,sent,glad,...,comfort,away,slave,someon,recogn,danger,back,independ,focu,awar
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
red_list_of_words = [list_of_words[x] for x in idx]

In [ ]:
dictionary = dict(zip(np.array(range(0,len(red_list_of_words))),red_list_of_words))
list(dictionary.items())[0:500]

In [ ]:
inv_dictionary = dict(zip(red_list_of_words,np.array(range(0,len(red_list_of_words)))))
inv_dictionary['u.s.']

1315

Removing unuseful words

In [ ]:
remove = np.array([
    inv_dictionary['steve'],
                   inv_dictionary['martin'],
                   inv_dictionary['nichola'],inv_dictionary['peter'],inv_dictionary['4'],
inv_dictionary['13'],
inv_dictionary['14'],
inv_dictionary['15'],
inv_dictionary['20'],inv_dictionary['2'],inv_dictionary['100'],
inv_dictionary['200'],
inv_dictionary['400'],
inv_dictionary['60'],
inv_dictionary['['],inv_dictionary['*'],
inv_dictionary['nut'],
inv_dictionary['book'],inv_dictionary['one'],inv_dictionary['robert'],
inv_dictionary['..'],
inv_dictionary['michael'],
inv_dictionary['c'],
inv_dictionary[']'],inv_dictionary['ms.'],inv_dictionary['u.s.'],
inv_dictionary['joseph'],
inv_dictionary['novel'],inv_dictionary['us'],inv_dictionary['t.'],
inv_dictionary['w.'],
inv_dictionary['...'],
inv_dictionary['i.e'],
inv_dictionary['%'],inv_dictionary['5'],
inv_dictionary['25'],inv_dictionary['s.'],
inv_dictionary['c.'],
inv_dictionary['john'],inv_dictionary['$'],inv_dictionary['1'],
inv_dictionary['40'],
inv_dictionary['90']
])

In [ ]:
d = da.drop(da.columns[remove], axis=1)
d.head()

,shallow,still,portray,experienc,serv,cours,son,sent,glad,manag,...,comfort,away,slave,someon,recogn,danger,back,independ,focu,awar
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Removing words with low IDF

In [ ]:
kk = []
for i in range(d.shape[1]) :
    kk.append(len(np.where(d.iloc[:,i] > 0)[0]))

In [ ]:
np_kk = np.asarray(kk)
np_kk.shape

(1470,)

In [ ]:
IDF_senza_log = np.asarray(np.repeat(2000,np_kk.shape[0])/np_kk)
IDF = np.log2(IDF_senza_log)

In [ ]:
IDF.max(),IDF.min()

(10.965784284662087, 0.9227570010675396)

In [ ]:
idx3 = np.asarray(np.where(IDF > 3)[0])
idx3.shape

(1415,)

In [ ]:
final = d.iloc[:,idx3]
final.head()

,shallow,still,portray,experienc,serv,cours,son,sent,glad,manag,...,comfort,away,slave,someon,recogn,danger,back,independ,focu,awar
0,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [ ]:
final_list_of_words = final.columns.values.tolist() # complete list of words
final_list_of_words

In [ ]:
col_da_tenere = final.columns.values
col_da_tenere

array(['shallow', 'insan', 'still', ..., 'crucial', 'focu', 'awar'],
      dtype=object)

In [ ]:
da_save = data_tfidf[:][col_da_tenere]
da_save.head()

,shallow,insan,still,circumst,portray,overview,experienc,serv,mountain,cours,...,someon,recogn,danger,senior,suppli,back,independ,crucial,focu,awar
0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.158117,0.0,0.0,0.113421,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
da_save.to_csv('reduced.csv', index=False)